In [1]:
!pip install numpy==1.21.6

     ---------------------------------------- 14.0/14.0 MB 6.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.21.6 which is incompatible.


In [2]:
!pip install CityLearn==2.1b9

     ---------------------------------------- 16.0/16.0 MB 9.8 MB/s eta 0:00:00
  Using cached gym-0.26.2.tar.gz (721 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 1.1/1.1 MB 5.0 MB/s eta 0:00:00
     ---------------------------------------- 75.1/75.1 kB 4.3 MB/s eta 0:00:00
     -------------------------------------- 162.6/162.6 MB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 10.0/10.0 MB 6.4 MB/s eta 0:00:00
  Using cached pytz-2023.3.post1-py2.py3-none-any.whl (502 kB)
  Using cached cloudpickle-2.2.1-py3-none-any.whl (25 kB)
  Using cached gym_notices-0.0.8-py3-none-any.whl (3.0 kB)
  Created wheel for gym: filename=gym-0.26.2-py3-n

In [3]:
!pip install gym==0.21.0


Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install Shimmy

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install pickle5

Defaulting to user installation because normal site-packages is not writeable


In [6]:
#!pip install agents

In [7]:
#!pip install rewards

In [8]:
!pip install stable_baselines3

Defaulting to user installation because normal site-packages is not writeable


In [9]:
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import numpy as np
import time
import os
import csv
from citylearn.citylearn import CityLearnEnv
from citylearn.reward_function import SolarPenaltyAndComfortReward, SolarPenaltyReward, ComfortReward

#from agents.user_agent import SubmissionAgent
#from rewards.user_reward import SubmissionReward
from stable_baselines3.sac import SAC
from stable_baselines3.ppo import PPO
from citylearn.wrappers import NormalizedObservationWrapper, StableBaselines3Wrapper

from tqdm import tqdm
import matplotlib.pyplot as plt 

C:\Users\gacece\AppData\Roaming\Python\Python37\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class WrapperEnv:
    """
    Env to wrap provide Citylearn Env data without providing full env
    Preventing attribute access outside of the available functions
    """
    def __init__(self, env_data):
        self.observation_names = env_data['observation_names']
        self.action_names = env_data['action_names']
        self.observation_space = env_data['observation_space']
        self.action_space = env_data['action_space']
        self.time_steps = env_data['time_steps']
        self.seconds_per_time_step = env_data['seconds_per_time_step']
        self.random_seed = env_data['random_seed']
        self.buildings_metadata = env_data['buildings_metadata']
        self.episode_tracker = env_data['episode_tracker']
    
    def get_metadata(self):
        return {'buildings': self.buildings_metadata}

def create_citylearn_env(config, reward_function):
    env = CityLearnEnv(config.SCHEMA, reward_function=reward_function)

    env_data = dict(
        observation_names = env.observation_names,
        action_names = env.action_names,
        observation_space = env.observation_space,
        action_space = env.action_space,
        time_steps = env.time_steps,
        random_seed = None,
        episode_tracker = None,
        seconds_per_time_step = None,
        buildings_metadata = env.get_metadata()['buildings']
    )

    wrapper_env = WrapperEnv(env_data)
    return env, wrapper_env


def update_power_outage_random_seed(env: CityLearnEnv, random_seed: int) -> CityLearnEnv:
    """Update random seed used in generating power outage signals.
    
    Used to optionally update random seed for stochastic power outage model in all buildings.
    Random seeds should be updated before calling :py:meth:`citylearn.citylearn.CityLearnEnv.reset`.
    """

    for b in env.buildings:
        b.stochastic_power_outage_model.random_seed = random_seed

    return env

In [3]:
class Config:
    data_dir = './data/'
    SCHEMA = os.path.join(data_dir, 'schemas/warm_up/schema.json')
    num_episodes = 1
    
config = Config()

In [4]:
env, wrapper_env = create_citylearn_env(config, ComfortReward)

In [5]:
env = NormalizedObservationWrapper(env)
env = StableBaselines3Wrapper(env)

# Test Trainning

In [6]:
agent = PPO('MlpPolicy', env)
agent.learn(total_timesteps=env.time_steps*1)

C:\Users\gacece\AppData\Roaming\Python\Python37\site-packages\stable_baselines3\common\vec_env\patch_gym.py:50: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  "You provided an OpenAI Gym environment. "


In [7]:
agent.save("models/SubmissionRewardPPO/SacModel_1epoch")

C:\Users\gacece\AppData\Roaming\Python\Python37\site-packages\stable_baselines3\common\save_util.py:278: UserWarning: Path 'models\SubmissionRewardPPO' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [8]:
agent = PPO('MlpPolicy', env)
agent.load("models/SubmissionRewardPPO/SacModel_1epoch")

# Real Trainning

In [9]:
agent = PPO('MlpPolicy', env)

In [11]:
#agent.load("models/ComfortReward/SacModel_38epoch")

for epochTrains in tqdm(range(40,80)):#changer le début du range si reprise d'entraienement
    agent.learn(total_timesteps=env.time_steps*2)
    agent.save("models/ComfortReward/SacModel_{}epoch".format(epochTrains*2))

  0%|          | 0/40 [00:00<?, ?it/s]C:\Users\gacece\AppData\Roaming\Python\Python37\site-packages\stable_baselines3\common\save_util.py:278: UserWarning: Path 'models\ComfortReward' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")
100%|██████████| 40/40 [15:21<00:00, 23.03s/it]


In [19]:
PathSave = "models/ComfortReward/"
trajectories_Full = []
episode_metrics = []
ind = 0

for model in tqdm(os.listdir(PathSave)):
    agent = PPO('MlpPolicy', env)
    if(model[:3]  == "Sac") :
        agent.load(PathSave+model[:-4])
        trajectories_Full.append([])
        observations = env.reset()
        agent_time_elapsed = 0

        step_start = time.perf_counter()
        actions = agent.predict(observations)
        agent_time_elapsed += time.perf_counter() - step_start

        episodes_completed = 0
        num_steps = 0
        interrupted = False

        while True:
            observations, rewards, done, _ = env.step(actions[0])
            if not done:
                step_start = time.perf_counter()
                actions = agent.predict(observations)

                agent_time_elapsed += time.perf_counter() - step_start
                trajectories_Full[ind].append([observations.tolist(), actions[0], rewards])
            else:
                episodes_completed += 1
                metrics_df = env.evaluate_citylearn_challenge()
                episode_metrics.append(metrics_df)
                #print(
                    #f"Episode complete: {episodes_completed} | Latest episode metrics: {metrics_df}"
                #)

                # Optional: Uncomment line below to update power outage random seed
                # from what was initially defined in schema
                env = update_power_outage_random_seed(env, 90000)

                observations = env.reset()

                step_start = time.perf_counter()
                actions = agent.predict(observations)
                agent_time_elapsed += time.perf_counter() - step_start

            num_steps += 1

            if episodes_completed >= 1:
                break

        ind += 1

100%|██████████| 40/40 [05:15<00:00,  7.89s/it]


In [16]:
import citylearn

In [17]:
print(citylearn.__version__)

2.1b9


In [ ]:
for i [0,100]:
    s = env.reset()
    a = agent.predict(s)
    MS,N,don = env.step(a)

# trajectoire 

In [20]:
rewardMean = []
ind = 0
for trajectoire in trajectories_Full:
    rewardMean.append(0)
    for step in trajectoire:
        rewardMean[ind] += step[2]
    rewardMean[ind] /= len(trajectoire)
    ind+=1

In [21]:
plt.clf()
plt.plot([k for k in range(len(trajectories_Full))], rewardMean, color="r")
plt.savefig(PathSave+"RewardMeanPerEpisode")

In [22]:
name = "trajectories_train_1_eval_1_reward_test3"

with open("trajectories/" + name + ".csv", "w", newline='') as fichier:
    writer = csv.writer(fichier)
    writer.writerow(["observations", "actions", "rewards"])
    for i in range(len(trajectories)):
        writer.writerow(trajectories[i])

FileNotFoundError: [Errno 2] No such file or directory: 'trajectories/trajectories_train_1_eval_1_reward_test3.csv'





# KPI






note:
    verbose
    tensorboard